# 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

In [1]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [2]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

In [198]:
X_train.head(n=50)

0     The Rock is destined to be the 21st Century 's...
1     The gorgeously elaborate continuation of `` Th...
2     Singer/composer Bryan Adams contributes a slew...
3                  Yet the act is still charming here .
4     Whether or not you 're enlightened by any of D...
5     Just the labour involved in creating the layer...
6     Part of the charm of Satin Rouge is that it av...
7     a screenplay more ingeniously constructed than...
8              `` Extreme Ops '' exceeds expectations .
9     Good fun , good action , good acting , good di...
10                     Dramas like this make it human .
11    Still , this flick is fun , and host to some t...
12    Australian actor/director John Polson and awar...
13    You walk out of The Good Girl with mixed emoti...
14          Absorbing character study by André Turpin .
15    If you love reading and/or poetry , then by al...
16                           You 'll probably love it .
17    `` Frailty '' has been written so well , t

In [191]:
y_train_labels

0       Positive
1       Positive
2       Positive
3       Positive
4       Positive
          ...   
6915    Negative
6916    Negative
6917    Positive
6918    Negative
6919    Negative
Name: label, Length: 6920, dtype: object

In [3]:
# This preprocessor function makes use of the tf.keras tokenizer
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


In [4]:
from tensorflow.keras.layers import Dense, Embedding,Flatten
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 16)            160000    
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 2)                 1282      
                                                                 
Total params: 161,282
Trainable params: 161,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
173/173 [==============================] - 2s 7ms/step - loss: 0.6664 - acc: 0.6134 - val_loss: 0.8460 - val_acc: 0.1488
Epoch 2/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6157 - acc: 0.6358 - val_loss: 0.7999 - val_acc: 0.2673
Epoch 3/10
173/173 [==============================

### Save preprocessor function to local "preprocessor.zip" file

In [5]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


### Save model to local ".onnx" file

In [6]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [7]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this specific Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [8]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [ ]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Your model has been submitted as model version 206

In [9]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=40))
model2.add(LSTM(32, return_sequences=True, dropout=0.2))
model2.add(LSTM(32, dropout=0.2))
model2.add(Flatten())
model2.add(Dense(2, activation='softmax'))

model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model2.fit(preprocessor(X_train), y_train,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.2)

173/173 [==============================] - 15s 61ms/step - loss: 0.6509 - acc: 0.6223 - val_loss: 0.7782 - val_acc: 0.4292


In [10]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
prediction_column_index=model2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Your model has been submitted as model version 214

In [11]:
# Compare two or more models 
data=mycompetition.compare_models([1, 2], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,Embedding,"[None, 40, 16]",160000.000000,Embedding,"[None, 40, 16]",160000
1,Flatten,"[None, 640]",0.000000,LSTM,"[None, 40, 32]",6272
2,Dense,"[None, 2]",1282.000000,LSTM,"[None, 32]",8320
3,None,None,nan,Flatten,"[None, 32]",0
4,None,None,nan,Dense,"[None, 2]",66


## Optional: Tune model within range of hyperparameters with Keras Tuner

In [12]:
! pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.4 MB/s eta 0:00:00


In [13]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [14]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
import keras_tuner as kt

#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Embedding(10000, 16, input_length=40))
    model.add(LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                   return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    model.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=1, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=1, validation_data=(preprocessor(x_val), y_val))


Trial 3 Complete [00h 02m 26s]
val_accuracy: 0.6488439440727234

Best val_accuracy So Far: 0.6683526039123535
Total elapsed time: 00h 03m 02s


In [15]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=1)

217/217 [==============================] - 33s 114ms/step - loss: 0.6691 - accuracy: 0.5789


In [16]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Your model has been submitted as model version 216


In [ ]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

# Models

This dataset has positive and negative labels assigned to sentences extracted from movie reviews based on the sentiment/ words/ language of the sentences/reviews. The data is split into training and testing datasets,X_train, X_test, y train, ytest to develop the model.

A predictive model will automatically segregate positive and negative movie reviews using sentiment analysis. It is faster and more efficient which saves time and money.

Once the model learns when reviews\ classify as positive or negative, it reduces a lot of work for organisations that primarily work in the film industry like give movie scores eg. rotten tomatoes. If they know whether a movie has more positive or negative reviews, it becomes easier to score them and leave a final review. 

## Model I 5 LSTM layers

In [19]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

modelI = Sequential()
modelI.add(Embedding(10000, 16, input_length=40))
modelI.add(LSTM(32, return_sequences=True, dropout=0.2))
modelI.add(LSTM(32, return_sequences=True))
modelI.add(LSTM(32, return_sequences=True))
modelI.add(LSTM(32, return_sequences=True))
modelI.add(LSTM(32, dropout=0.2))
modelI.add(Flatten())
modelI.add(Dense(2, activation='softmax'))

modelI.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = modelI.fit(preprocessor(X_train), y_train,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.2)

173/173 [==============================] - 31s 120ms/step - loss: 0.6658 - acc: 0.6134 - val_loss: 0.8745 - val_acc: 0.1611


In [20]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelI = model_to_onnx(modelI, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("modelI.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
#Submit Model I: 

#-- Generate predicted  values 
prediction_column_index=modelI.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model I to Competition Leaderboard
mycompetition.submit_model(model_filepath = "modelI.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Your model has been submitted as model version 402

## Model II 6 conv 1d layers

In [ ]:
# Use 1D Conv layer rather than RNN or LSTM or GRU to fit model
# Why? Much lighter model to fit. Here we are training on the full dataset.  If you try
# to build a model using LSTM code after running this one it will be much slower.

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

modelII = Sequential()
modelII.add(Embedding(10000, 16, input_length=40))
modelII.add(layers.Conv1D(64, 2, activation='relu')) 
modelII.add(layers.Conv1D(64, 2, activation='relu')) 
modelII.add(layers.Conv1D(32, 2, activation='relu'))
modelII.add(layers.Conv1D(32, 2, activation='relu'))  
modelII.add(layers.MaxPooling1D(2))
modelII.add(layers.Conv1D(16, 2, activation='relu')) 
modelII.add(layers.Conv1D(16, 2, activation='relu')) 
modelII.add(layers.GlobalAveragePooling1D())
modelII.add(Dense(2, activation='softmax'))

modelII.summary()


In [45]:
modelII.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = modelII.fit(preprocessor(X_train), y_train,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.2)

173/173 [==============================] - 5s 20ms/step - loss: 0.6677 - acc: 0.6149 - val_loss: 0.8862 - val_acc: 0.1488


In [46]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelII = model_to_onnx(modelII, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("modelII.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [47]:
#Submit Model II: 

#-- Generate predicted y values (Model 2)
prediction_column_index=modelII.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "modelII.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 1s 8ms/step
Insert search tags to help users find your model (optional): .
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 405

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


Your model has been submitted as model version 405

## Model III

In [83]:
# Download Glove embedding matrix weights 
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-18 17:10:30--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-18 17:10:31--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-18 17:10:31--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [86]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [85]:
import os
# Extract embedding data for 100 feature embedding matrix
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [114]:
# Build embedding matrix
embedding_dim = 100 
max_words = 10000 
word_index = tokenizer.word_index
maxlen=40
sequences = tokenizer.texts_to_sequences(X_train)

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [130]:
# Set up same model architecture as before and then import Glove weights to Embedding layer:
modelIII = Sequential()
modelIII.add(Embedding(max_words, embedding_dim,  input_length=maxlen))
modelIII.add(layers.Conv1D(64, 2, activation='relu')) 
modelIII.add(layers.Conv1D(64, 2, activation='relu')) 
modelIII.add(layers.Conv1D(32, 2, activation='relu'))
modelIII.add(layers.Conv1D(32, 2, activation='relu'))  
modelIII.add(layers.MaxPooling1D(2))
modelIII.add(layers.Conv1D(16, 2, activation='relu')) 
modelIII.add(layers.Conv1D(16, 2, activation='relu')) 
modelIII.add(layers.GlobalAveragePooling1D())
modelIII.add(Dense(2, activation='softmax'))



In [123]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.27085999,  0.044006  , -0.02026   , ..., -0.4923    ,
         0.63687003,  0.23642001],
       ...,
       [ 0.0075511 ,  0.068286  ,  0.51406997, ..., -0.6893    ,
         0.13776   ,  0.15468   ],
       [ 0.58331001,  0.76283997, -0.29846001, ..., -0.14741001,
         0.34698999,  0.26282999],
       [-0.060389  ,  0.59951001,  0.26980001, ...,  0.65445   ,
        -0.74105   ,  0.57037002]])

In [131]:
# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
modelIII.layers[0].set_weights([embedding_matrix])
modelIII.layers[0].trainable = False



modelIII.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

In [137]:
history = modelIII.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.01)
modelIII.save_weights('pre_trained_glove_model.h5')


# Training data small to speed up training. Increase for better fit.

Epoch 1/10
215/215 [==============================] - 5s 21ms/step - loss: 0.3313 - acc: 0.8600 - val_loss: 0.5351 - val_acc: 0.7429
Epoch 2/10
215/215 [==============================] - 5s 22ms/step - loss: 0.3027 - acc: 0.8727 - val_loss: 0.3406 - val_acc: 0.8714
Epoch 3/10
215/215 [==============================] - 4s 17ms/step - loss: 0.2805 - acc: 0.8838 - val_loss: 0.3504 - val_acc: 0.8714
Epoch 4/10
215/215 [==============================] - 3s 14ms/step - loss: 0.2501 - acc: 0.8952 - val_loss: 0.3352 - val_acc: 0.8857
Epoch 5/10
215/215 [==============================] - 3s 14ms/step - loss: 0.2281 - acc: 0.9089 - val_loss: 0.3196 - val_acc: 0.9286
Epoch 6/10
215/215 [==============================] - 4s 18ms/step - loss: 0.2080 - acc: 0.9159 - val_loss: 0.3871 - val_acc: 0.8429
Epoch 7/10
215/215 [==============================] - 5s 22ms/step - loss: 0.1779 - acc: 0.9279 - val_loss: 0.5771 - val_acc: 0.7143
Epoch 8/10
215/215 [==============================] - 4s 20ms/step - 

In [138]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelIII = model_to_onnx(modelIII, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("modelIII.onnx", "wb") as f:
    f.write(onnx_modelIII.SerializeToString())

In [140]:
#Submit Model III: 

#-- Generate predicted y values.
prediction_column_index=modelIII.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "modelIII.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 1s 9ms/step
Insert search tags to help users find your model (optional): .
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 432

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


 Your model has been submitted as model version 426, 425 

In [ ]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

## My best model was the one in which I used transfer learning with glove embeddings. I applied these weights on the conv 1 d model which had 2 layers of 64 neurons, 2 layers of 32 neurons and eventually 2 layers of 16 neurons. I ran 10 epochs, had a batch_size=32,validation_split=0.01, optimizer='rmsprop' and had an accuracy of 79.69%, an f-1 score of79.66%,, precision of 79.89% and recall of 79.70%

## Model IV- LSTM with 20 epochs, validation_split=0.001 with tuning to get best hyperparameters.

In [143]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

modelIV = Sequential()
modelIV.add(Embedding(10000, 16, input_length=40))
modelIV.add(LSTM(32, return_sequences=True, dropout=0.2))
modelIV.add(LSTM(32, return_sequences=True))
modelIV.add(LSTM(32, return_sequences=True))
modelIV.add(LSTM(32, return_sequences=True))
modelIV.add(LSTM(32, dropout=0.2))
modelIV.add(Flatten())
modelIV.add(Dense(2, activation='softmax'))

modelIV.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = modelIV.fit(preprocessor(X_train), y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.001)

Epoch 1/20
217/217 [==============================] - 55s 163ms/step - loss: 0.6823 - acc: 0.5426 - val_loss: 0.8633 - val_acc: 0.2857
Epoch 2/20
217/217 [==============================] - 32s 148ms/step - loss: 0.5438 - acc: 0.7260 - val_loss: 0.6464 - val_acc: 0.7143
Epoch 3/20
217/217 [==============================] - 42s 195ms/step - loss: 0.4200 - acc: 0.8138 - val_loss: 0.8320 - val_acc: 0.2857
Epoch 4/20
217/217 [==============================] - 32s 146ms/step - loss: 0.3390 - acc: 0.8568 - val_loss: 1.5982 - val_acc: 0.2857
Epoch 5/20
217/217 [==============================] - 29s 133ms/step - loss: 0.2897 - acc: 0.8786 - val_loss: 0.7765 - val_acc: 0.5714
Epoch 6/20
217/217 [==============================] - 27s 123ms/step - loss: 0.2491 - acc: 0.9011 - val_loss: 0.6813 - val_acc: 0.5714
Epoch 7/20
217/217 [==============================] - 36s 166ms/step - loss: 0.2151 - acc: 0.9135 - val_loss: 0.4963 - val_acc: 0.5714
Epoch 8/20
217/217 [==============================] - 3

In [147]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelIV = model_to_onnx(modelIV, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("modelIV.onnx", "wb") as f:
    f.write(onnx_modelIV.SerializeToString())

In [149]:
#Submit Model IV: 

#-- Generate predicted y values 
prediction_column_index=modelIV.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model IV to Competition Leaderboard
mycompetition.submit_model(model_filepath = "modelIV.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 2s 29ms/step
Insert search tags to help users find your model (optional): .
Provide any useful notes about your model (optional): .

Your model has been submitted as model version 437

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


THe above model has an accuracy of 80.01%, an f-1 score of 80.98%, a precision of 81.24% and a recall of 81.01%. I ran it for 20 epochs, used rmsprop as the optimiser and had a validation split value of 0.001. It had 5 lstm layers

Hyperparameters

In [151]:
! pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [152]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [153]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
import keras_tuner as kt

#Define model structure & parameter search space with function
def build_model(hp):
    modelIV = keras.Sequential()
    modelIV.add(Embedding(10000, 16, input_length=40))
    modelIV.add(LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                   return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    modelIV.add(Flatten())
    modelIV.add(Dense(2, activation='softmax'))
    modelIV.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return modelIV

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=1, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=1, validation_data=(preprocessor(x_val), y_val))


Trial 3 Complete [00h 03m 40s]
val_accuracy: 0.5946531891822815

Best val_accuracy So Far: 0.6546242833137512
Total elapsed time: 00h 09m 38s


In [155]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_modelIV = build_model(best_hps[0])



In [156]:
# Fit with the entire dataset.
tuned_modelIV.fit(x=preprocessor(X_train), y=y_train, epochs=1)

217/217 [==============================] - 216s 970ms/step - loss: 0.6894 - accuracy: 0.5730


In [158]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelIV = model_to_onnx(tuned_modelIV, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_modelIV.onnx", "wb") as f:
    f.write(onnx_modelIV.SerializeToString())

In [160]:
#Submit Model IV: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_modelIV.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_modelIV.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 9s 148ms/step
Insert search tags to help users find your model (optional): .
Provide any useful notes about your model (optional): .

Your model has been submitted as model version 440

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


The model does not work as well after this with an accuracy of 65.20%, an f-1 score of	63.21%, precision of	69.47% and recall of	65.23%

## Model V Transfer Learning with 20 epochs and batch size 64

In [175]:
# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
modelIII.layers[0].set_weights([embedding_matrix])
modelIII.layers[0].trainable = False



modelIII.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

In [176]:
history = modelIII.fit(preprocessor(X_train), y_train,
                    epochs=20,
                    batch_size=64,
                    validation_split=0.01)
modelIII.save_weights('pre_trained_glove_model.h5')


# Training data small to speed up training. Increase for better fit.

Epoch 1/20
108/108 [==============================] - 6s 40ms/step - loss: 0.0460 - acc: 0.9870 - val_loss: 0.5660 - val_acc: 0.8571
Epoch 2/20
108/108 [==============================] - 4s 35ms/step - loss: 0.0273 - acc: 0.9908 - val_loss: 0.7359 - val_acc: 0.8571
Epoch 3/20
108/108 [==============================] - 2s 21ms/step - loss: 0.0345 - acc: 0.9907 - val_loss: 0.8386 - val_acc: 0.8286
Epoch 4/20
108/108 [==============================] - 2s 21ms/step - loss: 0.0270 - acc: 0.9915 - val_loss: 0.6775 - val_acc: 0.8429
Epoch 5/20
108/108 [==============================] - 2s 21ms/step - loss: 0.0326 - acc: 0.9914 - val_loss: 0.5977 - val_acc: 0.8714
Epoch 6/20
108/108 [==============================] - 2s 22ms/step - loss: 0.0337 - acc: 0.9889 - val_loss: 1.0032 - val_acc: 0.8000
Epoch 7/20
108/108 [==============================] - 4s 34ms/step - loss: 0.0346 - acc: 0.9915 - val_loss: 0.6683 - val_acc: 0.8571
Epoch 8/20
108/108 [==============================] - 4s 36ms/step - 

In [177]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelV = model_to_onnx(modelIII, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("modelV.onnx", "wb") as f:
    f.write(onnx_modelV.SerializeToString())

In [179]:
#Submit Model III: 

#-- Generate predicted y values.
prediction_column_index=modelIII.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "modelV.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 6ms/step
Insert search tags to help users find your model (optional): .
Provide any useful notes about your model (optional): .

Your model has been submitted as model version 448

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

## Model VI 7 LSTM layers with no dropout and adam optimizer, batch size =64, 20 epochs and 0.001 validation split

In [181]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

modelVI = Sequential()
modelVI.add(Embedding(10000, 16, input_length=40))
modelVI.add(LSTM(32, return_sequences=True, dropout=0))
modelVI.add(LSTM(32, return_sequences=True))
modelVI.add(LSTM(32, return_sequences=True))
modelVI.add(LSTM(32, return_sequences=True))
modelVI.add(LSTM(32, return_sequences=True))
modelVI.add(LSTM(32, return_sequences=True))
modelVI.add(LSTM(32, dropout=0))
modelVI.add(Flatten())
modelVI.add(Dense(2, activation='softmax'))

modelVI.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = modelVI.fit(preprocessor(X_train), y_train,
                    epochs=20,
                    batch_size=64,
                    validation_split=0.001)

Epoch 1/20
109/109 [==============================] - 47s 283ms/step - loss: 0.6923 - acc: 0.5199 - val_loss: 0.7562 - val_acc: 0.2857
Epoch 2/20
109/109 [==============================] - 25s 228ms/step - loss: 0.6126 - acc: 0.6511 - val_loss: 0.6341 - val_acc: 0.8571
Epoch 3/20
109/109 [==============================] - 31s 286ms/step - loss: 0.3895 - acc: 0.8264 - val_loss: 0.7425 - val_acc: 0.2857
Epoch 4/20
109/109 [==============================] - 30s 273ms/step - loss: 0.2449 - acc: 0.9019 - val_loss: 0.7927 - val_acc: 0.5714
Epoch 5/20
109/109 [==============================] - 23s 207ms/step - loss: 0.1650 - acc: 0.9377 - val_loss: 0.9012 - val_acc: 0.5714
Epoch 6/20
109/109 [==============================] - 30s 273ms/step - loss: 0.1329 - acc: 0.9550 - val_loss: 1.2925 - val_acc: 0.7143
Epoch 7/20
109/109 [==============================] - 31s 283ms/step - loss: 0.1037 - acc: 0.9657 - val_loss: 1.1936 - val_acc: 0.7143
Epoch 8/20
109/109 [==============================] - 2

In [182]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_modelVI = model_to_onnx(modelVI, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("modelVI.onnx", "wb") as f:
    f.write(onnx_modelVI.SerializeToString())

In [184]:
#Submit Model VI: 

#-- Generate predicted y values 
prediction_column_index=modelVI.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model IV to Competition Leaderboard
mycompetition.submit_model(model_filepath = "modelVI.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 2s 38ms/step
Insert search tags to help users find your model (optional): .
Provide any useful notes about your model (optional): .

Your model has been submitted as model version 451

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


## FINAL COMMENTS

TO summarize alL of my models,

Model 1:
5 LSTM layers with 32 neurons each, batch size=32, validation split = 0.2, 1 epoch, optimiser= rmsprop.
It has an accuracy of 50.82%, an f-1 score of	35.23%, precision of	75.19% and recall of	50.88%

MODEL 2:
MODEL WITH AN EMBEDDED LAYER AND CONV 1 D LAYERS.
It has 2 layers with 64 neurons, 2 layers 32 neurons, Max pooling, 2 layers 16 neurons, Global average pooling. It uses the optimizer rmsprop, has a batch size =32 and validation split = 0.2. This model has an accuracy of 49.95%, an f-1 score of	33.31%, precision of	24.97% and recall of	50.00%

Model 3:
My best model before discussion- I used transfer learning with glove embeddings. I applied these weights on the conv 1 d model which had 2 layers of 64 neurons, 2 layers of 32 neurons and eventually 2 layers of 16 neurons. I ran 10 epochs, had a batch_size=32,validation_split=0.01, optimizer='rmsprop' and had an accuracy of 79.69%, an f-1 score of79.66%,, precision of 79.89% and recall of 79.70%

Model 4:
My best model overall- It has 5 lstm layers,each layer has 32 neurons. I ran it for 20 epochs, it had a batch size of 32, validation split of 0.001 and used the optimizer rmsprop.This model had the highest scores and best results. 
It had an accuracy of 81.01%, an f-1 score of	80.98%, precision of	81.24% and recall of	81.01%. However, after using best parameters to build the model, we had an accuracy of 65.20%, an f-1 score of 63.21%, precision of 69.47% and recall of 65.23% 

Model 5:
This model is similar to the other transfer learning model- model3. Since that was my best model before discussion, I wanted to incorporate some changes to see how much better it could be. I increased the batch size from 32 to 64 and ran it for 20 epochs. It is a transfer learning model with glove embeddings and is applied to the same conv 1 d model(model 2) as model 3. This however reduces the model's perfromance with an accuracy score of 77.61%, an f-1 score of 77.58%, precision of 77.73% and recall of 77.60%

Model 6:
Since my previous LSTM model (Model 4) was my best model, I wanted to change a few aspects about the model and check if it was still as good. I changed the dropout from 0.2 to none. I added two more LSTM layers, ran it for 20 epochs, with a batch size of 64 and a validation split og 0.001. Surprisingly, it underperformed and had an accuracy score of75.41%, f-1 score of	75.41%, precision of 75.42% and recall of	75.41%.

## Link to github
https://github.com/arushisaranath/Movie-Review-Sentiment-Analysis-Model